Решение системы линейных уравнений методом Гаусса. 

Материалы к вопросу https://ru.stackoverflow.com/questions/1271321

In [1]:
import numpy as np


In [2]:
matrix = np.array([[3.8, 6.7, -1.2, 5.2], 
                   [6.4, 1.3, -2.7, 3.8], 
                   [2.4, -4.5, 3.5, -0.6]])

withZero = np.array([[1,0,0, 1],
                     [0,0,1, 2],
                     [0,1,0, 3]], dtype=float)

# Наивная реализация
Наивная реализация метода Гаусса приведения к треугольной форме. Сломается на матрицах, содержащих нуль на диагонали. Например, на матрице с такими коэффициентами:
```
array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])
```

Функция принимает на вход матрицу `(N+1)xN` - в последней колонке свободные члены. Функция меняет матрицу, переданную в аргументе, поэтому если хочется сохранить матрицу, то вызывать нужно с `np.copy`: `gaussFunc(matrix.copy())`

In [3]:
def makeTriangleNaive(matrix):
    # функция меняет матрицу через побочные эффекты
    # если вам нужно сохранить прежнюю матрицу, скопируйте её np.copy
    for nrow, row in enumerate(matrix):
        # nrow равен номеру строки
        # row содержит саму строку матрицы
        divider = row[nrow] # диагональный элемент
        # делим на диагональный элемент.
        row /= divider
        # теперь надо вычесть приведённую строку из всех нижележащих строчек
        for lower_row in matrix[nrow+1:]:
            factor = lower_row[nrow] # элемент строки в колонке nrow
            lower_row -= factor*row # вычитаем, чтобы получить ноль в колонке nrow
    # все строки матрицы изменились, в принципе, можно и не возвращать
    return matrix

In [4]:
makeTriangleNaive(matrix.copy())

array([[ 1.        ,  1.76315789, -0.31578947,  1.36842105],
       [-0.        ,  1.        ,  0.06800211,  0.49657354],
       [ 0.        ,  0.        ,  1.        ,  0.09309401]])

Для нахождения решения нужно привести матрицу коэффициентов к диагональному виду. Тогда в последнем столбце будет находиться решение.

In [5]:
def makeIdentity(matrix):
    # перебор строк в обратном порядке 
    for nrow in range(len(matrix)-1,0,-1):
        row = matrix[nrow]
        for upper_row in matrix[:nrow]:
            factor = upper_row[nrow]
            # вычитать строки не нужно, так как в row только два элемента отличны от 0:
            # в последней колонке и на диагонали
            
            # вычитание в последней колонке
            upper_row[-1] -= factor*row[-1]
            # вместо вычитания 1*factor просто обнулим коэффициент в соотвествующей колонке. 
            upper_row[nrow] = 0
    return matrix

In [6]:
m1 = makeTriangleNaive(np.copy(matrix))
m2 = makeIdentity(m1)
m2

array([[ 1.        ,  0.        ,  0.        ,  0.53344344],
       [-0.        ,  1.        ,  0.        ,  0.49024295],
       [ 0.        ,  0.        ,  1.        ,  0.09309401]])

После приведения к диагональному виду корни находятся в последнем столбце.

In [7]:
roots = m2[:,-1]
roots

array([0.53344344, 0.49024295, 0.09309401])

**Проверка решения**

Для проверки извлечём матрицу коэффициентов, умножим её справа на столбец корней и вычтем столбец свободных членов исходной матрицы: `Ax - b`. Результат должен оказаться близким к нулю.

In [8]:
coefs = matrix[:,:-1]
coefs

array([[ 3.8,  6.7, -1.2],
       [ 6.4,  1.3, -2.7],
       [ 2.4, -4.5,  3.5]])

In [9]:
# свободные члены в последнем столбце
b = matrix[:,-1]

In [10]:
np.matmul(coefs, roots.T) - b

array([ 0.00000000e+00, -4.44089210e-16, -2.22044605e-16])

**Решение СЛАУ одной функцией**

In [11]:
def gaussSolveNaive(A, b=None):
    """Решает систему линейных алгебраических уравнений Ax=b
    Если b is None, то свободные коэффициенты в последней колонке"""
    shape = A.shape
    assert len(shape) == 2, ("Матрица не двумерная", shape) # двумерная матрица
    A = A.copy()
    if b is not None:
        assert shape[0] == shape[1], ("Матрица не квадратная", shape)
        assert b.shape == (shape[0],), ("Размерность свободных членов не соответствует матрица", shape, b.shape)
        # добавляем свободные члены дополнительным столбцом
        A = np.c_[A, b]
    else:
        # Проверяем, что квадратная плюс столбец
        assert shape[0]+1 == shape[1], ("Неверный формат матрицы", shape)
    makeTriangleNaive(A)
    makeIdentity(A)
    return A[:,-1]

In [12]:
gaussSolveNaive(matrix)

array([0.53344344, 0.49024295, 0.09309401])

In [13]:
gaussSolveNaive(matrix[:,:3], matrix[:,3])

array([0.53344344, 0.49024295, 0.09309401])

Когда на диагонали встречается ноль, происходит деление на ноль. Оно не выбрасывается как исключение, вместо этого возвращается `nan`

In [14]:
gaussSolveNaive(withZero)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


array([nan, nan, nan])

# Решение методом Гаусса с выбором главного элемента

Для того, чтобы избежать проблем с делением на ноль, и вообще повысить устойчивость счета, используется метод Гаусса с выбором главного элемента.

В этом методе перед тем как делить на диагональный элемент среди всех строк, лежащих ниже, находится строка с максимальным по модулю элементом в нужной колонке.

In [15]:
def makeTrianglePivot(matrix):
    for nrow in range(len(matrix)):
        # nrow равен номеру строки
        # np.argmax возвращает номер строки с максимальным элементом в уменьшенной матрице
        # которая начинается со строки nrow. Поэтому нужно прибавить nrow к результату
        pivot = nrow + np.argmax(abs(matrix[nrow:, nrow]))
        if pivot != nrow:
            # swap
            # matrix[nrow], matrix[pivot] = matrix[pivot], matrix[nrow] - не работает.
            # нужно переставлять строки именно так, как написано ниже
            # matrix[[nrow, pivot]] = matrix[[pivot, nrow]]
            matrix[nrow], matrix[pivot] = matrix[pivot], np.copy(matrix[nrow])
        row = matrix[nrow]
        divider = row[nrow] # диагональный элемент
        if abs(divider) < 1e-10:
            # почти нуль на диагонали. Продолжать не имеет смысла, результат счёта неустойчив
            raise ValueError("Матрица несовместна")
        # делим на диагональный элемент.
        row /= divider
        # теперь надо вычесть приведённую строку из всех нижележащих строчек
        for lower_row in matrix[nrow+1:]:
            factor = lower_row[nrow] # элемент строки в колонке nrow
            lower_row -= factor*row # вычитаем, чтобы получить ноль в колонке nrow
    return matrix

In [16]:
makeTrianglePivot(np.array([[1,0,0,1],
                         [0,0,1,2],
                         [0,1,0,3]
                        ], dtype=float))

array([[1., 0., 0., 1.],
       [0., 1., 0., 3.],
       [0., 0., 1., 2.]])

In [17]:
def gaussSolvePivot(A, b=None):
    """Решает систему линейных алгебраических уравнений Ax=b
    Если b is None, то свободные коэффициенты в последней колонке"""
    shape = A.shape
    assert len(shape) == 2, ("Матрица не двумерная", shape) # двумерная матрица
    A = A.copy()
    if b is not None:
        assert shape[0] == shape[1], ("Матрица не квадратная", shape)
        assert b.shape == (shape[0],), ("Размерность свободных членов не соответствует матрица", shape, b.shape)
        # добавляем свободные члены дополнительным столбцом
        A = np.c_[A, b]
    else:
        # Проверяем, что квадратная плюс столбец
        assert shape[0]+1 == shape[1], ("Неверный формат матрицы", shape)
    makeTrianglePivot(A)
    makeIdentity(A)
    return A[:,-1]

In [18]:
gaussSolvePivot(matrix)

array([0.53344344, 0.49024295, 0.09309401])

# Пример матрица 100x100

В примере решается случайная система линейных уравнений с матрицей 100x100

In [19]:
N = 100
randomSle = np.random.rand(N, N)
randomV = np.random.rand(N)

Для начала решим "наивным" способом. Вероятность того, что на диагонали будет нуль, пренебрежимо мала.

In [20]:
randomRoots = gaussSolveNaive(randomSle, randomV)
randomRoots

array([ 2.11416715, -1.30746648, -0.65419556, -3.51254613,  2.13898311,
        3.22928076,  0.45769601,  2.43369704,  3.2711606 ,  0.14570868,
        0.68509975,  1.70555571, -1.05707612,  0.94090601, -0.87739547,
       -0.70399065, -0.15676476,  1.00909638, -2.39858522,  0.62249626,
       -1.77693207,  0.08585223,  0.21890165,  0.74606491, -0.61614036,
        2.74852471, -1.35145162, -0.32323147, -0.0898949 , -0.74780049,
       -1.34755001,  1.58825864,  0.56227854, -0.51789052, -2.28951741,
        0.4885966 , -0.33649543, -1.33082582, -2.26453721, -0.46520173,
        1.09681358,  0.37987709,  2.93641096, -0.22906293, -2.43658322,
        2.16352784, -2.02093504, -1.66095716, -0.44670522,  1.87099628,
        1.5777987 ,  1.69613135, -2.01005121, -0.6260992 ,  2.33762135,
        1.87510222,  0.00690166, -0.60377963, -1.47735452,  1.21772367,
        0.65785427,  1.99543894, -1.03656166,  1.24759644, -0.13939762,
       -0.70099348, -1.37818259, -0.82149614,  0.36777295,  1.31

In [21]:
randomRoots2 = gaussSolvePivot(randomSle, randomV)

Проверим решение: вычислим максимум модуля в разности `Ax-b`

In [22]:
diff = np.matmul(randomSle, randomRoots) - randomV
np.max(np.abs(diff))

9.844347559351263e-13

In [23]:
diff = np.matmul(randomSle, randomRoots2) - randomV
np.max(np.abs(diff))

1.9872992140790302e-14

В обоих случаях `Ax` практически равно `b` - корни найдены успешно. Но решение, найденное методом с выбором главного элемента, построило чуть более точное решение

Сравним найденное решение с решателем, который поставляется с `numpy`:

In [24]:
np_roots = np.linalg.solve(randomSle, randomV)
np.max(np.abs(np.matmul(randomSle, np_roots) - randomV))

1.532107773982716e-14

In [25]:
max(abs(randomRoots - np_roots)), max(abs(randomRoots2 - np_roots))

(2.374989094278135e-12, 5.81756864903582e-14)

Решения очень близкие. Встроенный решатель построил ещё более точное решение.

## Сравнение времени счёта

In [26]:
%timeit -n10 -r 5 gaussSolveNaive(randomSle, randomV)

18.7 ms ± 2.66 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [27]:
%timeit -n10 -r 5 gaussSolvePivot(randomSle, randomV)

18.6 ms ± 367 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [28]:
%timeit -n50 -r 7 np.linalg.solve(randomSle, randomV)

483 µs ± 39.1 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


Методы решения, написанные на чистом пайтоне, считают практически с одинаковой скоростью, и примерно в 40 раз медленее встроенного решателя. Ничего удивительного, встроенный решатель написан на Си.

Ниже представлен трюк, как можно приблизить скорость работы пайтоновского кода к Си-шному, если самые трудоёмкие части кода откомпилировать в машинный код компилятором `numba`

## Ускорение счёта

Для начала обобщим метод решения, выделив функции приведения к треугольному виду и к диагональному виду в параметры.

In [29]:
def generalGauss(A,b, triangleFn=makeTrianglePivot, identityFn=makeIdentity):
    """Решает систему линейных алгебраических уравнений Ax=b
    Если b is None, то свободные коэффициенты в последней колонке"""
    shape = A.shape
    assert len(shape) == 2, ("Матрица не двумерная", shape) # двумерная матрица
    A = A.copy()
    if b is not None:
        assert shape[0] == shape[1], ("Матрица не квадратная", shape)
        assert b.shape == (shape[0],), ("Размерность свободных членов не соответствует матрица", shape, b.shape)
        # добавляем свободные члены дополнительным столбцом
        A = np.c_[A, b]
    else:
        # Проверяем, что квадратная плюс столбец
        assert shape[0]+1 == shape[1], ("Неверный формат матрицы", shape)
    A = triangleFn(A)
    A = identityFn(A)
    return np.array([ r[-1] for r in A ])

Проверим, что решение не изменилось

In [30]:
max(abs(generalGauss(randomSle, randomV, triangleFn=makeTriangleNaive, identityFn=makeIdentity) - randomRoots))


0.0

In [31]:
import numba

Немного видоизменённый вариант функции `makeTrianglePivot`, адаптированный к возможностям компилятора `numba`.

Декоратор `numba.njit` предписывает транслировать функцию в чистый машинный код, который не обращается к интерпретатору пайтона. В общем случае это невозможно, но в данном случае у нас все вычисления идут только с `numpy`, а для этого пакета `numba` умеет вызывать Си-инетерфейсы для соответствующих операций - индексирования, присваивания, арифметики.

In [32]:
m = np.random.rand(3,4)
list(m)

[array([0.35315579, 0.96583615, 0.53465876, 0.79239702]),
 array([0.04410346, 0.04067085, 0.99929292, 0.50134466]),
 array([0.10887379, 0.86640567, 0.04542141, 0.32615126])]

In [33]:
@numba.njit
def fastMakeTrianglePivot(matrix):
    buf = np.zeros(matrix.shape[1])

    for nrow in range(len(matrix)):
        pivot = nrow + np.argmax(np.abs(matrix[nrow:, nrow]))
        if pivot != nrow:
            matrix[nrow], matrix[pivot] = matrix[pivot], np.copy(matrix[nrow])
        row = matrix[nrow]
        divider = row[nrow] # диагональный элемент
        if abs(divider) < 1e-10:
            raise ValueError("Матрица несовместна")
        row[nrow:] *= 1/divider
        row[nrow] = 1.0
        for lr in range(nrow+1, len(matrix)):
            lower_row = matrix[lr]
            factor = lower_row[nrow]
            np.multiply(factor, row, buf)
            lower_row -= buf
            # lower_row -= factor*row
            # factor = matrix[lr, nrow]
            # matrix[lr] -= factor*row
    return matrix

In [34]:
@numba.njit
def fastMakeIdentity(matrix):
    N = matrix.shape[0]
#     for nrow in range(len(matrix)-1,0,-1):
#         root = matrix[nrow, -1]
#         matrix[nrow:,-1] -= root*matrix[nrow:,nrow]
#         matrix[nrow:,nrow] = 0.0
#     return matrix

    matrix = matrix.T
    roots = matrix[-1]
    for nrow in range(N-1,0,-1):
        root = roots[nrow]
        column = matrix[nrow]
        roots[:nrow] -= root*column[:nrow]
        column[:nrow] = 0.0
        # roots[nrow] = root
        
    return matrix.T

Сначала проверим, насколько выросла скорость от замены функции приведения к треугольному виду на скомпилированную
Функцию вызываем два раза. В первом вызове jit-компилятор `numba` транслирует функцию `fastMakeTrianglePivot` в машинный код. Это долгая операция, поэтому результаты измерения времени будут недостоверными.

In [35]:
m1 = fastMakeTrianglePivot(np.random.rand(4,5))
m2 = fastMakeIdentity(m1.copy())
m1, m2

(array([[ 1.00000000e+00,  9.77753041e-02,  7.47142889e-01,
          4.63444137e-01,  9.15647911e-01],
        [ 0.00000000e+00,  1.00000000e+00,  1.15719251e-02,
          1.34530085e-01, -7.60750213e-02],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
          8.61089532e-02,  4.43786691e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00, -1.02261805e+02]]),
 array([[   1.        ,    0.        ,    0.        ,    0.        ,
           40.07040617],
        [   0.        ,    1.        ,    0.        ,    0.        ,
           13.57418043],
        [   0.        ,    0.        ,    1.        ,    0.        ,
            9.24944366],
        [   0.        ,    0.        ,    0.        ,    1.        ,
         -102.26180491]]))

In [36]:
fastRoots = generalGauss(randomSle, randomV, fastMakeTrianglePivot, fastMakeIdentity)
np.max(np.abs(fastRoots - randomRoots))

2.3101520696400257e-12

In [37]:
%timeit -n10 -r3 generalGauss(randomSle, randomV, fastMakeTrianglePivot)

4.93 ms ± 80.6 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


Благодаря компилятору время работы снизилось в 3 раза. Теперь заменим функцию приведения к диагональному виду на скомпилированную.

In [38]:
%timeit -n15 -r 5 generalGauss(randomSle, randomV, fastMakeTrianglePivot, fastMakeIdentity)

613 µs ± 49.8 µs per loop (mean ± std. dev. of 5 runs, 15 loops each)


Итого скорость выросла в 20 раз.

Проигрыш по сравнению с решателем на чистом Си/Фортране, меньше чем в 2 раза

## Не всё так радужно

Возьмём матрицу 1000 на 1000

In [39]:
N = 1000
randomSle = np.random.rand(N, N)
randomV = np.random.rand(N)

In [40]:
randomRoots = gaussSolveNaive(randomSle, randomV)
randomRoots2 = gaussSolvePivot(randomSle, randomV)
diffNaive = np.matmul(randomSle, randomRoots) - randomV
diffPivot = np.matmul(randomSle, randomRoots2) - randomV
np.max(np.abs(diffNaive)), np.max(np.abs(diffPivot))

(2.966151768646341e-10, 7.12874204111813e-13)

Сравним найденные решения с решателем, который поставляется с `numpy`:

In [41]:
np_roots = np.linalg.solve(randomSle, randomV)
np.max(np.abs(np.matmul(randomSle, np_roots) - randomV))

4.199973702156967e-13

In [42]:
max(abs(randomRoots - np_roots)), max(abs(randomRoots2 - np_roots))

(3.828928285543043e-10, 1.4619416788264061e-12)

Решения близкие, но расстояние до них уже больше, чем для случая 100 на 100. Встроенный решатель построил ещё более точное решение.

### Сравнение времени счёта

In [43]:
%timeit -n3 -r 1 gaussSolveNaive(randomSle, randomV)

2.55 s ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)


In [44]:
%timeit -n3 -r 1 gaussSolvePivot(randomSle, randomV)

2.56 s ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)


In [45]:
%timeit -n30 -r 7 np.linalg.solve(randomSle, randomV)

28.3 ms ± 2.69 ms per loop (mean ± std. dev. of 7 runs, 30 loops each)


Методы решения, написанные на чистом пайтоне, считают практически с одинаковой скоростью, но разрыв с Сишным/фортрановским решателем уже почти в тысячу раз. Причина в том, что метод Гаусса требует порядка $O(n^3)$ арифметических операций, и $O(n^2)$ операций аллокации временных векторов для вычитания. При увеличении размерности системы в 10 раз время счёта выросло в 100 раз. Можно предположить, что основной вклад в замедление счёта - это время на работу с памятью.

In [46]:
%timeit -n5 -r3 generalGauss(randomSle, randomV, fastMakeTrianglePivot)

712 ms ± 5.79 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


Благодаря компилятору время работы по-прежнему в 3 раза меньше, чем у функции на Python

In [47]:
%timeit -n10 -r 3 generalGauss(randomSle, randomV, fastMakeTrianglePivot, fastMakeIdentity)

278 ms ± 1.71 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


Выигрыш уже не столь значителен, меньше чем в 10 раз. И разница с встроенным решателем уже в 8 раз.